In [138]:
import csv
from sklearn import cluster
import numpy

In [147]:
%%time
values = {}
max_sim = 0.0

with open('/data/python-scrapy-data-mining/sims_hobi_mainan_action-figure.csv') as raw:
    for (p1, p2, sim) in csv.reader(raw):
        sim = float(sim)
        if max_sim < sim:
            max_sim = sim
        values[(int(p1), int(p2))] = sim

CPU times: user 1.4 s, sys: 4 ms, total: 1.4 s
Wall time: 1.64 s


In [141]:
prod_ids = sorted(list(set((p1 for (p1, _) in values.keys()))))
print "N:", len(prod_ids)

N: 2583


In [148]:
def get_item(i, j):
    i, j = prod_ids[i], prod_ids[j]
    if i > j:
        sim = values.get((j, i), 0)
    elif i == j:
        return 0.0
    else:
        sim = values.get((i, j), 0)
    return (max_sim - sim) / max_sim

In [149]:
arr = numpy.ndarray((len(prod_ids), len(prod_ids)))

In [150]:
%%time
for i in xrange(len(prod_ids)):
    for j in xrange(len(prod_ids)):
        arr[i, j] = get_item(i, j)

CPU times: user 7.35 s, sys: 100 ms, total: 7.45 s
Wall time: 7.44 s


In [180]:
db = cluster.DBSCAN(metric="precomputed", eps=0.4).fit(arr)
db.labels_

array([-1, -1, -1, ..., -1, -1, -1])

In [181]:
for i, l in sorted(enumerate(db.labels_), key=lambda e: -e[1]):
    print prod_ids[i], ":", l

61216 : 0
70148 : 0
70152 : 0
70219 : 0
154190 : 0
191503 : 0
214 : -1
931 : -1
1810 : -1
2942 : -1
5621 : -1
7955 : -1
7956 : -1
7957 : -1
7967 : -1
7968 : -1
7969 : -1
7970 : -1
7971 : -1
7972 : -1
7973 : -1
7974 : -1
7975 : -1
7976 : -1
7981 : -1
7982 : -1
7983 : -1
7984 : -1
7985 : -1
7986 : -1
7987 : -1
7994 : -1
8006 : -1
8007 : -1
8030 : -1
8031 : -1
8032 : -1
8049 : -1
8050 : -1
8054 : -1
8062 : -1
8071 : -1
8089 : -1
10084 : -1
10111 : -1
10114 : -1
10115 : -1
10117 : -1
10118 : -1
10119 : -1
10120 : -1
10121 : -1
10343 : -1
10345 : -1
10346 : -1
10348 : -1
10351 : -1
10398 : -1
10402 : -1
10405 : -1
10407 : -1
10408 : -1
10411 : -1
10412 : -1
10414 : -1
10415 : -1
10417 : -1
10430 : -1
11134 : -1
11146 : -1
11156 : -1
11160 : -1
11162 : -1
11164 : -1
11167 : -1
11168 : -1
11171 : -1
11174 : -1
11175 : -1
11181 : -1
11182 : -1
11183 : -1
11184 : -1
11190 : -1
11193 : -1
11198 : -1
11206 : -1
11230 : -1
11264 : -1
11275 : -1
14526 : -1
14562 : -1
15063 : -1
15205 : -1
15208 : -

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [156]:
db.labels_

array([-1, -1, -1, ..., -1, -1, -1])